# Motor Score analysis of PPMI 


In [1]:
import pandas as pd
import numpy as np
import os
import glob
import functools
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, accuracy_score
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

from sklearn.feature_selection import SelectFromModel
from sklearn.utils import class_weight

# In case you do not have following packages installed, uncomment instalisation.

#!pip3 install xgboost
#from xgboost import XGBClassifier


### Data to be loaded ###

In [2]:
path2 = Path("/scratch/znazari/PPMI_ver_sep2022/study_data/motor_score")# the path to the data



In [3]:
motor_score_1 = pd.read_csv(path2/"MDS-UPDRS_Part_I.csv",delimiter=',')
motor_score_1

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,NUPSOURC,NP1COG,NP1HALL,NP1DPRS,NP1ANXS,NP1APAT,NP1DDS,NP1RTOT,ORIG_ENTRY,LAST_UPDATE
0,272451201,3000,BL,NUPDRS1,02/2011,1.0,1,0,1,1,0,0,3.0,02/2011,2020-06-25 16:02:19.0
1,338701901,3000,V04,NUPDRS1,03/2012,1.0,0,0,1,0,0,0,1.0,03/2012,2020-06-25 16:02:21.0
2,385008801,3000,V06,NUPDRS1,02/2013,1.0,1,0,1,1,0,0,3.0,02/2013,2020-06-25 16:02:22.0
3,437130601,3000,V08,NUPDRS1,03/2014,1.0,1,0,0,2,0,0,3.0,05/2014,2020-06-25 16:02:22.0
4,512466501,3000,V10,NUPDRS1,03/2015,1.0,0,0,0,0,0,0,0.0,03/2015,2020-06-25 16:02:23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16543,71c489f2-480a-4d7d-963d-4be0bec724b9,160040,BL,NUPDRS1,07/2022,1.0,0,0,0,0,0,0,0.0,08/2022,2022-08-04 00:00:00.0
16544,a8b2df17-960c-497f-b77c-08a1ddee8fc4,160231,BL,NUPDRS1,08/2022,1.0,0,0,0,0,0,0,0.0,08/2022,2022-08-09 00:00:00.0
16545,c1313e5f-fa29-4319-a15f-2b21db5cd8a2,161236,BL,NUPDRS1,07/2022,1.0,1,0,0,0,0,0,1.0,08/2022,2022-08-11 00:00:00.0
16546,c28a3518-abb5-42d3-8865-579be8bb3713,162140,BL,NUPDRS1,08/2022,1.0,0,0,0,0,0,0,0.0,08/2022,2022-08-18 00:00:00.0


In [6]:
# Assuming 'motor_score_1' is your DataFrame

# Extract rows with 'BL' in the 'EVENT_ID' column
filtered_data = motor_score_1[motor_score_1['EVENT_ID'].str.contains('BL')]

# Select only the 'PATNO' and 'NP1RTOT' columns
columns_=['PATNO', 'NP1COG','NP1HALL','NP1DPRS','NP1ANXS','NP1APAT','NP1DDS']
result = filtered_data[columns_]
result


,PATNO,NP1COG,NP1HALL,NP1DPRS,NP1ANXS,NP1APAT,NP1DDS
0,3000,1,0,1,1,0,0
9,3001,0,0,0,0,0,0
28,3002,1,0,1,0,1,0
47,3003,0,0,0,1,0,0
64,3004,0,0,0,0,0,0
...,...,...,...,...,...,...,...
16543,160040,0,0,0,0,0,0
16544,160231,0,0,0,0,0,0
16545,161236,1,0,0,0,0,0
16546,162140,0,0,0,0,0,0


In [9]:
# reading the file which contains diagnosis
path3=Path("/scratch/znazari/PPMI_ver_sep2022/study_data/Subject_Characteristics/")
diago=pd.read_csv(path3/"Participant_Status.csv")

In [9]:
diago

,PATNO,COHORT,COHORT_DEFINITION,ENROLL_DATE,ENROLL_STATUS,STATUS_DATE,ENROLL_AGE,INEXPAGE,AV133STDY,PPMI_ONLINE_ENROLL,...,COMMENTS,CONDATE,ENRLPINK1,ENRLPRKN,ENRLSRDC,ENRLHPSM,ENRLRBD,ENRLLRRK2,ENRLSNCA,ENRLGBA
0,3000,2,Healthy Control,02/2011,enrolled,05/2021,69.1,INEXHC,0.0,NO,...,NaN,06/2021,0.0,0.0,0.0,0,0,0,0,0
1,3001,1,Parkinson's Disease,03/2011,enrolled,09/2021,65.1,INEXPD,0.0,NO,...,NaN,06/2021,0.0,0.0,1.0,0,0,0,0,0
2,3002,1,Parkinson's Disease,03/2011,enrolled,09/2021,67.6,INEXPD,0.0,NO,...,NaN,06/2021,0.0,0.0,1.0,0,0,0,0,0
3,3003,1,Parkinson's Disease,04/2011,enrolled,01/2022,56.7,INEXPD,0.0,NO,...,NaN,06/2021,0.0,0.0,1.0,0,0,0,0,0
4,3004,2,Healthy Control,04/2011,enrolled,01/2022,59.4,INEXHC,0.0,YES,...,NaN,06/2021,0.0,0.0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,162994,1,Parkinson's Disease,NaN,screened,07/2022,NaN,INEXPD,0.0,NO,...,NaN,NaN,0.0,0.0,1.0,0,0,0,0,0
2347,163265,1,Parkinson's Disease,NaN,screened,07/2022,NaN,INEXPD,0.0,YES,...,NaN,NaN,0.0,0.0,1.0,0,0,0,0,0
2348,164900,1,Parkinson's Disease,NaN,screened,07/2022,NaN,INEXPD,0.0,NO,...,NaN,NaN,0.0,0.0,1.0,0,0,0,0,0
2349,167222,1,Parkinson's Disease,NaN,screened,08/2022,NaN,INEXPD,0.0,NO,...,NaN,NaN,0.0,0.0,1.0,0,0,0,0,0


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder


# Merge the DataFrames based on the "PATNO" column
merged_df = pd.merge(result, diago[['PATNO', 'COHORT_DEFINITION']], on='PATNO', how='inner')

# Filter rows with only "Parkinson's Disease" or "Healthy Control"
filtered_df = merged_df[merged_df['COHORT_DEFINITION'].isin(["Parkinson's Disease", 'Healthy Control'])]

# Extract features (X) and target variable (y)
X = filtered_df[columns_]
X_=X.set_index('PATNO')
#y = (filtered_df['COHORT_DEFINITION'] == "Parkinson's Disease").astype(int)

y = filtered_df['COHORT_DEFINITION']

# Convert diagnosis labels to numeric values using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_, y_encoded, test_size=0.2, random_state=42)

# Define and train the XGBoost model
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calculate the AUC score
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f'AUC Score: {auc_score}')


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:NP1COG: object, NP1HALL: object, NP1DPRS: object, NP1ANXS: object, NP1APAT: object, NP1DDS: object

In [20]:
X_=X.set_index('PATNO')
_.columns

Index(['NP1COG', 'NP1HALL', 'NP1DPRS', 'NP1ANXS', 'NP1APAT', 'NP1DDS'], dtype='object')

In [12]:
y

0           Healthy Control
1       Parkinson's Disease
2       Parkinson's Disease
3       Parkinson's Disease
4           Healthy Control
               ...         
1876    Parkinson's Disease
1877    Parkinson's Disease
1878    Parkinson's Disease
1879    Parkinson's Disease
1881    Parkinson's Disease
Name: COHORT_DEFINITION, Length: 1186, dtype: object

In [6]:
motor_score_2 = pd.read_csv(path2/"MDS_UPDRS_Part_II__Patient_Questionnaire.csv",delimiter=',')
motor_score_2

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,NUPSOURC,NP2SPCH,NP2SALV,NP2SWAL,NP2EAT,...,NP2HWRT,NP2HOBB,NP2TURN,NP2TRMR,NP2RISE,NP2WALK,NP2FREZ,NP2PTOT,ORIG_ENTRY,LAST_UPDATE
0,272451701,3000,BL,NUPDRS2P,02/2011,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,02/2011,2020-06-25 16:02:19.0
1,338702901,3000,V04,NUPDRS2P,03/2012,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,03/2012,2020-06-25 16:02:22.0
2,385009601,3000,V06,NUPDRS2P,02/2013,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,02/2013,2020-06-25 16:02:22.0
3,437131001,3000,V08,NUPDRS2P,03/2014,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,05/2014,2020-06-25 16:02:22.0
4,512467901,3000,V10,NUPDRS2P,03/2015,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,03/2015,2020-06-25 16:02:23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16564,66c0ebc7-c64c-429a-ad8c-a32507be75b5,160040,BL,NUPDRSP,07/2022,1,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,08/2022,2022-08-04 00:00:00.0
16565,c6916457-e5fa-47f0-bed9-250c70200062,160231,BL,NUPDRSP,07/2022,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,08/2022,2022-08-09 00:00:00.0
16566,d7cdaa2b-03cb-4ab5-8070-29d92a865480,161236,BL,NUPDRSP,07/2022,1,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,08/2022,2022-08-11 00:00:00.0
16567,084df822-0092-4728-a057-3158707dfc0d,162140,BL,NUPDRSP,08/2022,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,08/2022,2022-08-18 00:00:00.0


In [8]:
motor_score_3 = pd.read_csv(path2/"MDS_UPDRS_Part_III.csv",delimiter=',', low_memory=False)
motor_score_3

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,PDMEDDT,PDMEDTM,PDSTATE,EXAMTM,DBS_STATUS,...,DYSKIRAT,NHY,PDTRTMNT,DBSONTM,DBSOFFTM,HRPOSTMED,HRDBSOFF,HRDBSON,ORIG_ENTRY,LAST_UPDATE
0,272451901,3000,BL,NUPDRS3,02/2011,NaN,NaN,NaN,13:17:00,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,02/2011,2020-06-25 16:02:19.0
1,338703101,3000,V04,NUPDRS3,03/2012,NaN,NaN,NaN,13:47:00,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,03/2012,2020-06-25 16:02:22.0
2,385009801,3000,V06,NUPDRS3,02/2013,NaN,NaN,NaN,12:22:00,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,02/2013,2020-06-25 16:02:22.0
3,437131401,3000,V08,NUPDRS3,03/2014,NaN,NaN,NaN,13:22:00,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,05/2014,2020-06-25 16:02:22.0
4,512469901,3000,V10,NUPDRS3,03/2015,NaN,NaN,NaN,11:43:00,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,03/2015,2020-06-25 16:02:23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18998,6f71db91-a468-4362-8a2f-c0abc5d3bc9f,160231,BL,NUPDRS3,08/2022,NaN,NaN,NaN,NaN,NaN,...,0.0,2,0.0,NaN,NaN,NaN,NaN,NaN,08/2022,2022-08-09 00:00:00.0
18999,4bf2b126-4606-4ef0-8f4b-020e213a27a3,161236,BL,NUPDRS3,07/2022,NaN,NaN,NaN,NaN,NaN,...,0.0,1,0.0,NaN,NaN,NaN,NaN,NaN,08/2022,2022-08-11 00:00:00.0
19000,41851079-7a2e-4079-9f58-5c6a3a8ceffd,162140,BL,NUPDRS3,08/2022,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,08/2022,2022-08-18 00:00:00.0
19001,8f879af8-dd3a-4c40-9578-9aae9f13782f,162929,BL,NUPDRS3,08/2022,NaN,NaN,NaN,NaN,NaN,...,0.0,2,0.0,NaN,NaN,NaN,NaN,NaN,08/2022,2022-08-19 00:00:00.0


In [9]:
motor_score_4 = pd.read_csv(path2/"MDS-UPDRS_Part_IV__Motor_Complications.csv",delimiter=',', low_memory=False)
motor_score_4

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,NP4WDYSK,NP4WDYSKDEN,NP4WDYSKNUM,NP4WDYSKPCT,NP4DYSKI,...,NP4OFFPCT,NP4FLCTI,NP4FLCTX,NP4DYSTN,NP4DYSTNDEN,NP4DYSTNNUM,NP4DYSTNPCT,NP4TOT,ORIG_ENTRY,LAST_UPDATE
0,6a7cf647-eead-4890-8514-cb68e5d0cd47,3001,R17,NUPDRS4,11/2021,1,4.0,16.0,25.0,0,...,0.0,0,0,0,0.0,0.0,0.0,1.0,12/2021,2021-12-02 00:00:00.0
1,473773301,3001,V09,NUPDRS4,11/2014,0,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,NaN,NaN,0.0,11/2014,2020-06-25 16:04:34.0
2,516499401,3001,V10,NUPDRS4,04/2015,0,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,NaN,NaN,0.0,04/2015,2020-06-25 16:04:35.0
3,588306101,3001,V12,NUPDRS4,06/2016,0,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,NaN,NaN,0.0,09/2016,2020-06-25 16:04:35.0
4,681769601,3001,V14,NUPDRS4,03/2018,1,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,NaN,NaN,1.0,04/2018,2020-06-25 16:04:36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6994,3af1305a-d029-420f-b6fe-0454f0b7ac4b,127741,V02,NUPDRS4,07/2022,0,0.0,7.0,17.0,0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,07/2022,2022-07-20 00:00:00.0
6995,055ba1c2-434b-4afb-b711-4e533802a6bd,136600,V02,NUPDRS4,07/2022,0,0.0,17.0,0.0,0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,07/2022,2022-07-15 00:00:00.0
6996,42ec6d0f-12a4-404f-b8a8-b1448e32393e,137264,V02,NUPDRS4,07/2022,0,0.0,19.0,0.0,0,...,0.0,0,0,0,0.0,19.0,0.0,0.0,07/2022,2022-07-14 00:00:00.0
6997,0ee05714-7e57-4e1b-baf2-53e9f7acf4ff,137424,V02,NUPDRS4,07/2022,0,0.0,18.0,0.0,0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,07/2022,2022-07-15 00:00:00.0


In [10]:
motor_score_5 = pd.read_csv(path2/"Participant_Motor_Function_Questionnaire.csv",delimiter=',', low_memory=False)
motor_score_5

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,CMPLBY2,TRBUPCHR,WRTSMLR,VOICSFTR,POORBAL,FTSTUCK,LSSXPRSS,ARMLGSHK,TRBBUTTN,SHUFFLE,MVSLOW,TOLDPD,ORIG_ENTRY,LAST_UPDATE
0,44c8070b-e3bc-4fb7-9a8a-6257ff64b099,3000,V17,PQUEST,05/2021,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,06/2021,2021-06-01 00:00:00.0
1,78a9c4bb-f275-4803-950f-4c14329f38f1,3001,V17,PQUEST,09/2021,1,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,01/2022,2022-01-06 00:00:00.0
2,7727fc05-ff14-479d-b8ad-70ec21e923fc,3001,V18,PQUEST,07/2022,1,1.0,1.0,1.0,2.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,07/2022,2022-07-08 00:00:00.0
3,69aa855f-963d-46b2-974a-69a934721125,3002,V17,PQUEST,09/2021,1,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,01/2022,2022-01-06 00:00:00.0
4,368f1e99-c8ab-4202-b066-cd0e55b2e359,3002,V18,PQUEST,03/2022,1,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,03/2022,2022-03-11 00:00:00.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1952,acc904f3-cbaf-49f2-8f30-49b81a4c7f9c,160040,BL,PQUEST,07/2022,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,08/2022,2022-08-04 00:00:00.0
1953,e57ee4f4-7660-43ee-a3d0-cf62800867b5,160231,BL,PQUEST,07/2022,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,08/2022,2022-08-09 00:00:00.0
1954,c458167f-7506-4bc2-9c48-88e9d177b438,161236,BL,PQUEST,07/2022,1,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,1.0,08/2022,2022-08-11 00:00:00.0
1955,d60548f4-ce03-4fb1-ac9d-aaa7bc3a4dc7,162140,BL,PQUEST,08/2022,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,08/2022,2022-08-18 00:00:00.0


In [4]:
proteomic = pd.read_csv(path3/"Project_151_pQTL_in_CSF_1_of_7_Batch_Corrected_.csv",delimiter=',')
proteomic

,PATNO,SEX,COHORT,CLINICAL_EVENT,TYPE,TESTNAME,TESTVALUE,UNITS,PLATEID,RUNDATE,PROJECTID,PI_NAME,PI_INSTITUTION,update_stamp
0,53595,Female,PD,BL,Cerebrospinal Fluid,5632-6_3,10.182013,log2 RFU,P0022899,2019-08-04,151,David Stone,Merck,2022-05-13 10:38:41.0
1,53595,Female,PD,BL,Cerebrospinal Fluid,5631-83_3,6.346285,log2 RFU,P0022899,2019-08-04,151,David Stone,Merck,2022-05-13 10:38:41.0
2,53595,Female,PD,BL,Cerebrospinal Fluid,5630-48_3,12.943830,log2 RFU,P0022899,2019-08-04,151,David Stone,Merck,2022-05-13 10:38:41.0
3,53595,Female,PD,BL,Cerebrospinal Fluid,5629-58_3,5.891402,log2 RFU,P0022899,2019-08-04,151,David Stone,Merck,2022-05-13 10:38:41.0
4,53595,Female,PD,BL,Cerebrospinal Fluid,5628-21_3,12.836350,log2 RFU,P0022899,2019-08-04,151,David Stone,Merck,2022-05-13 10:38:41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803875,3825,Male,PD,BL,Cerebrospinal Fluid,12908-15_3,6.522282,log2 RFU,P0022899,2019-08-04,151,David Stone,Merck,2022-05-13 10:38:08.0
803876,3825,Male,PD,BL,Cerebrospinal Fluid,12906-137_3,6.020084,log2 RFU,P0022899,2019-08-04,151,David Stone,Merck,2022-05-13 10:38:08.0
803877,3825,Male,PD,BL,Cerebrospinal Fluid,12904-180_3,4.182774,log2 RFU,P0022899,2019-08-04,151,David Stone,Merck,2022-05-13 10:38:08.0
803878,3825,Male,PD,BL,Cerebrospinal Fluid,12900-29_3,7.283812,log2 RFU,P0022899,2019-08-04,151,David Stone,Merck,2022-05-13 10:38:08.0


In [5]:
# Specify the base file name and path
base_file_name = "Project_151_pQTL_in_CSF_{}_of_7_Batch_Corrected_.csv"

# Number of files
num_files = 7

# List to store DataFrames
dfs = []

# Loop through the file indices and read each file
for file_index in range(1, num_files + 1):
    file_name = base_file_name.format(file_index)
    file_path = path3 / file_name
    
    # Check if the file exists before attempting to read it
    if file_path.is_file():
        # Read the CSV file and append it to the list
        df = pd.read_csv(file_path, delimiter=',')
        dfs.append(df)
    else:
        print(f"File {file_name} not found.")


In [8]:
dfs[6]

,PATNO,SEX,COHORT,CLINICAL_EVENT,TYPE,TESTNAME,TESTVALUE,UNITS,PLATEID,RUNDATE,PROJECTID,PI_NAME,PI_INSTITUTION,update_stamp
0,59649,Male,PD,BL,Cerebrospinal Fluid,5632-6_3,11.026966,log2 RFU,P0024868,2019-02-12,151,David Stone,Merck,2022-05-13 10:38:41.0
1,59649,Male,PD,BL,Cerebrospinal Fluid,5631-83_3,6.609041,log2 RFU,P0024868,2019-02-12,151,David Stone,Merck,2022-05-13 10:38:41.0
2,59649,Male,PD,BL,Cerebrospinal Fluid,5630-48_3,13.466670,log2 RFU,P0024868,2019-02-12,151,David Stone,Merck,2022-05-13 10:38:41.0
3,59649,Male,PD,BL,Cerebrospinal Fluid,5629-58_3,5.835645,log2 RFU,P0024868,2019-02-12,151,David Stone,Merck,2022-05-13 10:38:41.0
4,59649,Male,PD,BL,Cerebrospinal Fluid,5628-21_3,12.769220,log2 RFU,P0024868,2019-02-12,151,David Stone,Merck,2022-05-13 10:38:41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722530,3917,Male,Control,BL,Cerebrospinal Fluid,9745-20_3,6.931300,log2 RFU,P0024869,2019-02-12,151,David Stone,Merck,2022-05-13 10:39:16.0
722531,3917,Male,Control,BL,Cerebrospinal Fluid,9747-48_3,7.043459,log2 RFU,P0024869,2019-02-12,151,David Stone,Merck,2022-05-13 10:39:16.0
722532,3917,Male,Control,BL,Cerebrospinal Fluid,9748-31_3,8.461362,log2 RFU,P0024869,2019-02-12,151,David Stone,Merck,2022-05-13 10:39:16.0
722533,3917,Male,Control,BL,Cerebrospinal Fluid,9749-190_3,9.049630,log2 RFU,P0024869,2019-02-12,151,David Stone,Merck,2022-05-13 10:39:16.0


In [6]:
# Concatenate all DataFrames into a single DataFrame
result_df = pd.concat(dfs, ignore_index=True)

# Filter out patients diagnosed as Prodromal
result_df = result_df[result_df['COHORT'] != 'Prodromal']

# Pivot the DataFrame to get the desired format
result_pivot = result_df.pivot(index='TESTNAME', columns='PATNO', values='TESTVALUE')



In [20]:
result_pivot

PATNO,3000,3001,3002,3003,3004,3006,3007,3008,3009,3010,...,74067,74199,74817,75409,75421,75480,75484,75505,75524,75562
TESTNAME,,,,,,,,,,,,,,,,,,,,,
10000-28_3,7.016730,7.454336,7.247439,7.010630,6.877898,6.842311,7.754820,7.001725,6.820676,7.104007,...,6.899428,6.816018,6.853592,6.781720,6.924048,6.758358,6.870476,6.844687,7.029326,6.666132
10001-7_3,7.870548,8.121843,8.344479,7.222104,7.650171,7.092575,7.806605,8.196070,7.769967,7.359637,...,7.997572,7.275574,7.427730,7.834432,8.015048,7.337290,7.599946,7.499072,7.612910,7.258944
10003-15_3,6.147252,6.222071,6.065235,6.138153,6.203758,6.221370,6.174415,6.171208,6.280763,6.188014,...,6.324074,6.192135,6.248550,6.312524,6.219102,6.054284,6.384542,6.236579,6.343984,6.186476
10006-25_3,6.371610,6.418159,6.365775,6.357083,6.361865,6.183732,6.238306,6.332403,6.393692,6.567999,...,6.303177,6.577943,6.329391,6.163935,6.480985,6.350333,6.355463,6.292218,6.339972,6.344696
10008-43_3,5.882283,5.934242,5.827745,5.914232,5.893985,6.054652,5.939613,6.070155,5.952001,5.913274,...,5.988861,5.892367,5.974385,5.883404,5.922584,6.049692,5.950037,5.970764,5.944815,5.865619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993-11_3,6.230929,6.209682,6.586607,6.225973,6.257445,6.412115,7.233072,5.979002,6.252459,6.274228,...,6.218115,6.537651,6.495690,6.300791,6.372259,6.215987,6.386449,6.247486,6.541717,6.430384
9994-217_3,7.031689,7.129638,7.345001,7.330257,7.413496,7.024051,6.918511,6.947370,7.070061,7.149448,...,7.076340,7.068358,7.308743,7.017250,6.983826,7.071633,7.193699,7.390031,7.071616,7.576549
9995-6_3,9.630040,9.827683,9.805028,8.835570,9.094281,8.520557,9.484474,9.514471,9.189601,9.000802,...,9.627819,9.281438,9.397977,9.215428,9.766957,8.772783,9.377158,9.155109,9.443181,9.159902


In [24]:
result_pivot_filtered = result_pivot.drop(columns=mutation_patients['PATNO'], errors='ignore')

In [25]:
result_pivot_filtered

PATNO,3000,3001,3002,3003,3004,3006,3007,3008,3009,3010,...,4121,4122,4123,4124,4125,4126,4135,4136,4139,41410
TESTNAME,,,,,,,,,,,,,,,,,,,,,
10000-28_3,7.016730,7.454336,7.247439,7.010630,6.877898,6.842311,7.754820,7.001725,6.820676,7.104007,...,6.996939,6.838948,6.935341,6.938593,6.743564,6.879410,7.047316,6.894967,6.768009,6.610080
10001-7_3,7.870548,8.121843,8.344479,7.222104,7.650171,7.092575,7.806605,8.196070,7.769967,7.359637,...,8.012181,7.855806,8.035597,8.196864,7.766080,7.773577,7.825966,7.479678,7.780084,7.602446
10003-15_3,6.147252,6.222071,6.065235,6.138153,6.203758,6.221370,6.174415,6.171208,6.280763,6.188014,...,6.320647,6.416003,6.275744,6.149764,6.164363,6.323608,6.161733,6.171370,6.253532,6.240810
10006-25_3,6.371610,6.418159,6.365775,6.357083,6.361865,6.183732,6.238306,6.332403,6.393692,6.567999,...,6.348242,6.437493,6.301871,6.312332,6.423467,6.403437,6.363899,6.400867,6.433781,6.435529
10008-43_3,5.882283,5.934242,5.827745,5.914232,5.893985,6.054652,5.939613,6.070155,5.952001,5.913274,...,5.958741,6.001704,5.896644,6.025008,5.837845,5.918820,5.846805,6.181146,5.925006,5.976737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993-11_3,6.230929,6.209682,6.586607,6.225973,6.257445,6.412115,7.233072,5.979002,6.252459,6.274228,...,6.164821,6.167718,6.148084,6.120723,6.348303,6.229989,6.162735,6.375987,6.529382,6.481852
9994-217_3,7.031689,7.129638,7.345001,7.330257,7.413496,7.024051,6.918511,6.947370,7.070061,7.149448,...,7.016618,7.318267,7.170244,7.052944,7.128567,7.139254,7.292240,7.036760,7.025085,7.114605
9995-6_3,9.630040,9.827683,9.805028,8.835570,9.094281,8.520557,9.484474,9.514471,9.189601,9.000802,...,9.602305,9.572287,9.585365,9.752996,9.752487,9.334244,9.307760,9.011721,9.608131,8.949228


In [7]:
file_path = "/home/znazari/data/open_proteomic/patients_proteomics.csv"

# Write the DataFrame to a CSV file
result_pivot.to_csv(file_path, index=True)

In [26]:
mutation_patients = pd.read_csv(path2/"mutated_g_patients.csv")
mutation_patients

,PATNO
0,40012
1,40067
2,40163
3,40273
4,40338
...,...
758,142394
759,142848
760,148092
761,150110


In [8]:
patient_diagnosis_df = result_df[['PATNO', 'COHORT']].drop_duplicates()

In [38]:
patient_diagnosis_df

,PATNO,COHORT
0,53595,PD
4785,3029,Control
9570,3963,PD
14355,3316,Control
19140,3124,PD
...,...,...
5512320,3812,Control
5517105,3817,Control
5521890,3792,PD
5526675,3793,PD


In [9]:
file_path2 = "/home/znazari/data/open_proteomic/patients_diagnosis.csv"
patient_diagnosis_df.to_csv(file_path2, index=False)

In [39]:
new_patients_diagnos = patient_diagnosis_df[~patient_diagnosis_df['PATNO'].isin(mutation_patients['PATNO'])]


In [40]:
new_patients_diagnos

,PATNO,COHORT
4785,3029,Control
9570,3963,PD
14355,3316,Control
19140,3124,PD
23925,3175,PD
...,...,...
5512320,3812,Control
5517105,3817,Control
5521890,3792,PD
5526675,3793,PD


In [44]:
# Assuming you have two DataFrames: result_pivot and patient_diagnosis_df

#  Merge the two DataFrames on 'PATNO'
merged_df = pd.merge(result_pivot_filtered.T, new_patients_diagnos, on='PATNO')

# Separate features (X) and target variable (y)
X = merged_df.drop(['PATNO', 'COHORT'], axis=1)
y = merged_df['COHORT']

# Convert diagnosis labels to numeric values using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

#  Initialize and train the XGBoost model
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

#  Make predictions on the testing set
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

# Calculate AU-ROC score
au_roc_score = roc_auc_score(y_test, y_pred_proba)

# Print the AU-ROC score
print(f'AU-ROC Score: {au_roc_score}')



AU-ROC Score: 0.7596714361420243


In [45]:
# Assuming you have two DataFrames: result_pivot and patient_diagnosis_df

#  Merge the two DataFrames on 'PATNO'
merged_df = pd.merge(result_pivot.T, patient_diagnosis_df, on='PATNO')

# Separate features (X) and target variable (y)
X = merged_df.drop(['PATNO', 'COHORT'], axis=1)
y = merged_df['COHORT']

# Convert diagnosis labels to numeric values using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

#  Initialize and train the XGBoost model
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

#  Make predictions on the testing set
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

# Calculate AU-ROC score
au_roc_score = roc_auc_score(y_test, y_pred_proba)

# Print the AU-ROC score
print(f'AU-ROC Score: {au_roc_score}')



AU-ROC Score: 0.7742836547125608


In [ ]:

# Merge the two DataFrames on 'PATNO'
merged_df = pd.merge(result_pivot.T, patient_diagnosis_df, on='PATNO')

# Separate features (X) and target variable (y)
X = merged_df.drop(['PATNO', 'COHORT'], axis=1)
y = merged_df['COHORT']

# Convert diagnosis labels to numeric values using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Initialize StratifiedKFold for cross-validation
n_splits = 10  # Number of folds
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize an array to store AU-ROC scores
au_roc_scores = []

# Perform cross-validation and repeat XGBoost algorithm
for _ in range(5):  # Repeat 5 times
    for train_index, test_index in skf.split(X, y_encoded):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]

        # Initialize and train the XGBoost model
        xgb_model = XGBClassifier()
        xgb_model.fit(X_train, y_train)

        # Make predictions on the testing set
        y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

        # Calculate AU-ROC score
        au_roc_score = roc_auc_score(y_test, y_pred_proba)

        # Append the AU-ROC score to the array
        au_roc_scores.append(au_roc_score)

# Calculate the average AU-ROC score over all repetitions and folds
average_au_roc_score = sum(au_roc_scores) / len(au_roc_scores)

# Print the average AU-ROC score
print(f'Average AU-ROC Score: {average_au_roc_score}')
